# VishwamAI Model Architecture Analysis

This notebook provides a comprehensive analysis of the VishwamAI model architecture, with focus on:
1. Architecture Components
2. TPU Optimizations
3. Attention Mechanisms
4. Memory Efficiency
5. Performance Characteristics

In [1]:
# Import required libraries
import jax
import jax.numpy as jnp
import flax.linen as nn
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Dict, List, Tuple, Any, Optional

from vishwamai.model import VishwamAI
from vishwamai.kernels.kernel import fp8_gemm_optimized
from vishwamai.layers.attention import FlashAttention
from vishwamai.transformer import (
    TransformerModel,
    EnhancedTransformerModel,
    create_vishwamai_transformer
)
from vishwamai.profiler import TPUProfiler

2025-03-20 16:26:02.189408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742468162.208229   33269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742468162.213656   33269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## 1. Model Configurations

VishwamAI supports multiple model sizes optimized for different use cases:

1. Base Model (768M parameters)
2. Large Model (1.5B parameters)
3. XL Model (3B parameters)
4. XXL Model (7B parameters)

Let's analyze different configurations:

In [2]:
def create_model_configs():
    """Create different model size configurations"""
    configs = {
        'base': {
            'vocab_size': 32000,
            'num_layers': 12,
            'num_heads': 12,
            'head_dim': 64,
            'hidden_dim': 768,
            'mlp_dim': 3072,
            'max_seq_len': 2048
        },
        'large': {
            'vocab_size': 32000,
            'num_layers': 24,
            'num_heads': 16,
            'head_dim': 96,
            'hidden_dim': 1536,
            'mlp_dim': 6144,
            'max_seq_len': 2048
        },
        'xl': {
            'vocab_size': 32000,
            'num_layers': 32,
            'num_heads': 24,
            'head_dim': 128,
            'hidden_dim': 2048,
            'mlp_dim': 8192,
            'max_seq_len': 2048
        },
        'xxl': {
            'vocab_size': 32000,
            'num_layers': 40,
            'num_heads': 32,
            'head_dim': 128,
            'hidden_dim': 4096,
            'mlp_dim': 16384,
            'max_seq_len': 2048
        }
    }
    
    # Add common configurations
    for config in configs.values():
        config.update({
            'dropout_rate': 0.1,
            'use_enhanced': True,
            'use_rotary': True,
            'use_flash_attn': True,
            'use_rms_norm': True,
            'dtype': 'bfloat16'
        })
    
    return configs

## 2. Architecture Analysis

VishwamAI implements several key optimizations:

1. **Flash Attention**: Memory-efficient attention implementation
2. **RMSNorm**: Faster alternative to LayerNorm
3. **Rotary Position Embeddings**: Better positional encoding
4. **TPU-Optimized Linear Layers**: Using fp8_gemm_optimized
5. **Mixed Precision Training**: Using bfloat16

In [3]:
def analyze_architecture(config: Dict[str, Any]) -> Dict[str, Any]:
    """Analyze model architecture components and memory usage"""
    
    def calculate_memory(params: int, dtype: str = 'bfloat16') -> float:
        """Calculate memory usage in GB"""
        bytes_per_param = 2 if dtype == 'bfloat16' else 4
        return (params * bytes_per_param) / (1024 ** 3)
    
    # Calculate parameters per component
    h = config['hidden_dim']
    v = config['vocab_size']
    l = config['num_layers']
    m = config['mlp_dim']
    
    embedding_params = v * h
    attention_params = l * (4 * h * h)  # QKV + output
    ffn_params = l * (2 * h * m)  # Two linear layers
    norm_params = l * 2 * h  # Two norms per layer
    
    total_params = embedding_params + attention_params + ffn_params + norm_params
    
    # Memory analysis
    memory_analysis = {
        'total_params': total_params,
        'params_gb': calculate_memory(total_params),
        'activation_gb': calculate_memory(config['max_seq_len'] * h * 4),  # Rough estimate
        'attention_gb': calculate_memory(config['max_seq_len']**2 * config['num_heads']),
        'components': {
            'embedding': embedding_params,
            'attention': attention_params,
            'ffn': ffn_params,
            'norm': norm_params
        }
    }
    
    # Theoretical throughput (tokens/second)
    # Based on documented TPU v2/v3 performance characteristics
    tflops = 420  # TPU v3-8 peak TFLOPS
    flops_per_token = total_params * 2  # Forward + backward
    memory_analysis['theoretical_throughput'] = (tflops * 1e12) / flops_per_token
    
    return memory_analysis

In [4]:
# Analyze all configurations
configs = create_model_configs()
analyses = {name: analyze_architecture(config) 
           for name, config in configs.items()}

# Print summary
for name, analysis in analyses.items():
    print(f"\n=== {name.upper()} Model Analysis ===")
    print(f"Total Parameters: {analysis['total_params']:,}")
    print(f"Model Size (GB): {analysis['params_gb']:.2f}")
    print(f"Peak Activation Memory (GB): {analysis['activation_gb']:.2f}")
    print(f"Peak Attention Memory (GB): {analysis['attention_gb']:.2f}")
    print(f"Theoretical Throughput: {analysis['theoretical_throughput']:,.0f} tokens/sec")
    
    # Component breakdown
    print("\nParameter Distribution:")
    for component, params in analysis['components'].items():
        percentage = 100 * params / analysis['total_params']
        print(f"{component:>10}: {percentage:6.2f}%")


=== BASE Model Analysis ===
Total Parameters: 109,529,088
Model Size (GB): 0.20
Peak Activation Memory (GB): 0.01
Peak Attention Memory (GB): 0.09
Theoretical Throughput: 1,917,299 tokens/sec

Parameter Distribution:
 embedding:  22.44%
 attention:  25.85%
       ffn:  51.70%
      norm:   0.02%

=== LARGE Model Analysis ===
Total Parameters: 728,702,976
Model Size (GB): 1.36
Peak Activation Memory (GB): 0.02
Peak Attention Memory (GB): 0.12
Theoretical Throughput: 288,183 tokens/sec

Parameter Distribution:
 embedding:   6.75%
 attention:  31.08%
       ffn:  62.16%
      norm:   0.01%

=== XL Model Analysis ===
Total Parameters: 1,676,279,808
Model Size (GB): 3.12
Peak Activation Memory (GB): 0.03
Peak Attention Memory (GB): 0.19
Theoretical Throughput: 125,277 tokens/sec

Parameter Distribution:
 embedding:   3.91%
 attention:  32.03%
       ffn:  64.06%
      norm:   0.01%

=== XXL Model Analysis ===
Total Parameters: 8,184,463,360
Model Size (GB): 15.24
Peak Activation Memory (GB

## 3. TPU Optimizations

Key TPU optimizations in VishwamAI:

1. **Memory Layout**
   - Aligned tensor dimensions for TPU cores
   - Efficient sharding across TPU matrix units
   - Optimized memory access patterns

2. **Computation Optimizations**
   - fp8_gemm_optimized for matrix operations
   - Flash Attention for O(n) memory complexity
   - Fused operations where possible

3. **Data Pipeline**
   - Efficient data loading and prefetching
   - Optimized input processing
   - Smart batching strategies

In [5]:
def benchmark_tpu_performance(config: Dict[str, Any]) -> Dict[str, float]:
    """Benchmark TPU performance characteristics"""
    # Initialize model
    model = create_vishwamai_transformer(config)
    
    # Setup profiler
    profiler = TPUProfiler({'log_dir': 'tpu_profiles'})
    
    # Generate sample batch
    batch_size = 32
    seq_len = 512
    x = jnp.ones((batch_size, seq_len), dtype=jnp.int32)
    
    # Initialize model parameters
    rng = jax.random.PRNGKey(0)
    variables = model.init(rng, x)
    
    # Warmup
    with profiler.profile_region('warmup'):
        _ = model.apply(variables, x, train=False)
    
    # Benchmark forward pass
    times = []
    for _ in range(10):
        start = time.time()
        _ = model.apply(variables, x, train=False)
        jax.tree_util.tree_map(lambda x: x.block_until_ready(), _)
        times.append(time.time() - start)
    
    # Calculate metrics
    forward_latency = np.mean(times)
    tokens_per_second = (batch_size * seq_len) / forward_latency
    
    # Get TPU utilization from profiler
    metrics = profiler.get_metrics_summary()
    tpu_utilization = metrics.get('tpu_utilization_mean', 0.0)
    memory_efficiency = metrics.get('memory_efficiency_mean', 0.0)
    
    return {
        'forward_latency': forward_latency,
        'tokens_per_second': tokens_per_second,
        'tpu_utilization': tpu_utilization,
        'memory_efficiency': memory_efficiency
    }

In [6]:
# Benchmark base model
base_perf = benchmark_tpu_performance(configs['base'])

print("\n=== TPU Performance Analysis (Base Model) ===")
print(f"Forward Pass Latency: {base_perf['forward_latency']*1000:.2f}ms")
print(f"Throughput: {base_perf['tokens_per_second']:,.0f} tokens/sec")
print(f"TPU Utilization: {base_perf['tpu_utilization']*100:.1f}%")
print(f"Memory Efficiency: {base_perf['memory_efficiency']*100:.1f}%")

2025-03-20 16:29:37.586880: W external/xla/xla/hlo/transforms/simplifiers/hlo_rematerialization.cc:3021] Can't reduce memory use below 999.61MiB (1048170986 bytes) by rematerialization; only reduced to 1.95GiB (2097152000 bytes), down from 1.95GiB (2097152000 bytes) originally
2025-03-20 16:29:47.587566: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.95GiB (rounded to 2097152000)requested by op 
2025-03-20 16:29:47.587833: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] ******************************************__________________________________________________________
2025-03-20 16:29:47.587566: W external/xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.95GiB (rounded to 2097152000)requested by op 
2025-03-20 16:29:47.587833: W external/xla/xla/tsl/framework/bfc_allocator.cc:512] ******************************************_________________________________

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 2097152000 bytes.

## 4. Memory Analysis and Optimization Strategy

VishwamAI implements several memory optimization techniques:

1. **Gradient Checkpointing**
   - Trades computation for memory
   - Configurable checkpointing frequency

2. **Activation Recomputation**
   - Selective recomputation of activations
   - Smart caching strategies

3. **Attention Optimization**
   - Flash Attention for linear memory scaling
   - Efficient KV cache management
   - Smart attention patterns

4. **Mixed Precision**
   - bfloat16 for compute
   - float32 for accumulation
   - Selective precision control

In [ ]:
# Memory efficiency analysis function
def analyze_memory_efficiency(config: Dict[str, Any]) -> Dict[str, float]:
    """Analyze memory efficiency of different components"""
    seq_len = config['max_seq_len']
    hidden_dim = config['hidden_dim']
    num_heads = config['num_heads']
    batch_size = 32
    
    # Calculate theoretical memory requirements
    activation_memory = batch_size * seq_len * hidden_dim * 2  # bfloat16
    attention_memory = batch_size * num_heads * seq_len * seq_len * 2
    gradient_memory = activation_memory  # Roughly equal for simple backprop
    
    # Calculate optimized memory with techniques enabled
    flash_attention_memory = batch_size * num_heads * seq_len * 2  # O(n) vs O(n^2)
    gradient_checkpointing_memory = activation_memory / 4  # Rough estimate with optimal checkpointing
    mixed_precision_memory = {
        'compute': activation_memory,  # bfloat16
        'accumulation': gradient_memory * 2  # float32 for stability
    }
    
    efficiency_metrics = {
        'baseline_memory': activation_memory + attention_memory + gradient_memory,
        'optimized_memory': flash_attention_memory + gradient_checkpointing_memory + mixed_precision_memory['compute'],
        'memory_savings': 1.0 - (flash_attention_memory + gradient_checkpointing_memory) / (activation_memory + attention_memory),
        'techniques': {
            'flash_attention_savings': 1.0 - (flash_attention_memory / attention_memory),
            'gradient_checkpoint_savings': 1.0 - (gradient_checkpointing_memory / activation_memory),
            'mixed_precision_ratio': mixed_precision_memory['compute'] / mixed_precision_memory['accumulation']
        }
    }
    
    return efficiency_metrics

In [ ]:
# Add visualization of memory usage across different configurations
def plot_memory_analysis(analyses):
    """Plot memory usage and efficiency metrics"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Memory breakdown
    models = list(analyses.keys())
    metrics = ['params_gb', 'activation_gb', 'attention_gb']
    x = np.arange(len(models))
    width = 0.25
    
    for i, metric in enumerate(metrics):
        values = [analyses[model][metric] for model in models]
        ax1.bar(x + i*width, values, width, label=metric.replace('_gb', ''))
    
    ax1.set_ylabel('Memory (GB)')
    ax1.set_title('Memory Usage Breakdown')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(models)
    ax1.legend()
    
    # Component distribution
    for i, model in enumerate(models):
        components = analyses[model]['components']
        sizes = list(components.values())
        labels = list(components.keys())
        ax2.pie(sizes, labels=labels, autopct='%1.1f%%',
                startangle=90, radius=0.8 + i*0.2)
    
    ax2.set_title('Parameter Distribution by Component')
    plt.tight_layout()
    plt.show()

In [ ]:
# Add performance analysis section
def analyze_performance(config: Dict[str, Any], batch_sizes: List[int] = [1, 8, 32, 128]):
    """Analyze performance characteristics across different batch sizes"""
    results = []
    
    for batch_size in batch_sizes:
        # Make a copy of config to avoid modifying the original
        test_config = config.copy()
        test_config['batch_size'] = batch_size  # Add batch_size to config
        
        # Run benchmark with the modified config
        perf = benchmark_tpu_performance(test_config)
        results.append({
            'batch_size': batch_size,
            'throughput': perf['tokens_per_second'],
            'latency': perf['forward_latency'],
            'efficiency': perf['tpu_utilization']
        })
    
    return results

In [ ]:
def plot_performance_analysis(perf_results: List[Dict[str, Any]]):
    """Plot performance analysis results"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    batch_sizes = [r['batch_size'] for r in perf_results]
    throughputs = [r['throughput'] for r in perf_results]
    latencies = [r['latency'] * 1000 for r in perf_results]  # Convert to ms
    
    ax1.plot(batch_sizes, throughputs, 'o-')
    ax1.set_xlabel('Batch Size')
    ax1.set_ylabel('Throughput (tokens/sec)')
    ax1.set_title('Throughput vs Batch Size')
    ax1.set_xscale('log', base=2)
    ax1.set_yscale('log', base=2)
    ax1.grid(True)
    
    ax2.plot(batch_sizes, latencies, 'o-')
    ax2.set_xlabel('Batch Size')
    ax2.set_ylabel('Latency (ms)')
    ax2.set_title('Latency vs Batch Size')
    ax2.set_xscale('log', base=2)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Run analysis for base model
print("\n=== Memory Efficiency Analysis (Base Model) ===")
base_memory = analyze_memory_efficiency(configs['base'])
print(f"Memory Savings: {base_memory['memory_savings']*100:.1f}%")
print("\nTechnique Breakdown:")
for technique, saving in base_memory['techniques'].items():
    print(f"{technique}: {saving*100:.1f}% improvement")

# Plot analyses
plot_memory_analysis(analyses)

# Run performance analysis
print("\n=== Performance Analysis (Base Model) ===")
perf_results = analyze_performance(configs['base'])
plot_performance_analysis(perf_results)

# Conclusions and model capabilities
model_capabilities = {
    'base': {
        'recommended_batch_size': 32,
        'max_seq_length': 2048,
        'typical_throughput': base_perf['tokens_per_second'],
        'memory_requirement': analyses['base']['params_gb'],
        'optimal_use_cases': [
            'Fine-tuning on single TPU',
            'Low-latency inference',
            'Memory-constrained environments'
        ]
    },
    'xl': {
        'recommended_batch_size': 16,
        'max_seq_length': 2048,
        'typical_throughput': analyses['xl']['theoretical_throughput'],
        'memory_requirement': analyses['xl']['params_gb'],
        'optimal_use_cases': [
            'Large-scale training',
            'Complex reasoning tasks',
            'Multi-TPU training'
        ]
    }
}

print("\n=== Model Capabilities Summary ===")
for model_size, caps in model_capabilities.items():
    print(f"\n{model_size.upper()} Model:")
    print(f"Recommended batch size: {caps['recommended_batch_size']}")
    print(f"Max sequence length: {caps['max_seq_length']}")
    print(f"Typical throughput: {caps['typical_throughput']:,.0f} tokens/sec")
    print(f"Memory requirement: {caps['memory_requirement']:.1f} GB")
    print("Optimal use cases:")
    for use_case in caps['optimal_use_cases']:
        print(f"- {use_case}")

In [ ]:
# Model Architecture Analysis
def analyze_architecture(config):
    """Analyze model architecture and attention patterns"""
    
    # Architecture components
    architecture = {
        'embedding_dim': config['hidden_size'],
        'num_layers': config['num_hidden_layers'],
        'num_heads': config['num_attention_heads'],
        'head_dim': config['hidden_size'] // config['num_attention_heads'],
        'feed_forward_dim': config['intermediate_size'],
        'vocab_size': config['vocab_size']
    }
    
    # Attention analysis
    attention_stats = {
        'head_capacity': architecture['head_dim'],
        'total_attention_capacity': architecture['head_dim'] * config['num_attention_heads'],
        'attention_bottleneck_ratio': architecture['head_dim'] / architecture['embedding_dim'],
        'heads_per_layer': config['num_attention_heads']
    }
    
    # Depth analysis
    depth_metrics = {
        'depth_to_width_ratio': config['num_hidden_layers'] / (config['hidden_size'] / 64),
        'computational_depth': config['num_hidden_layers'] * 2,  # Self-attention + FFN
        'parameter_efficiency': config['hidden_size'] * config['num_hidden_layers'] / (config['vocab_size'] * config['hidden_size'])
    }
    
    return {
        'architecture': architecture,
        'attention': attention_stats,
        'depth': depth_metrics
    }

def plot_attention_patterns(arch_analysis):
    """Visualize attention patterns and architecture characteristics"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot attention head distribution
    heads_per_layer = arch_analysis['attention']['heads_per_layer']
    head_dim = arch_analysis['attention']['head_capacity']
    layers = range(arch_analysis['architecture']['num_layers'])
    
    attention_capacity = np.ones(len(layers)) * heads_per_layer * head_dim
    ax1.plot(layers, attention_capacity, 'b-', label='Total Attention Capacity')
    ax1.fill_between(layers, attention_capacity, alpha=0.3)
    ax1.set_xlabel('Layer')
    ax1.set_ylabel('Attention Capacity')
    ax1.set_title('Attention Capacity Distribution')
    ax1.legend()
    ax1.grid(True)
    
    # Plot architecture dimensions
    dims = {
        'Embedding': arch_analysis['architecture']['embedding_dim'],
        'Head': arch_analysis['attention']['head_capacity'],
        'FFN': arch_analysis['architecture']['feed_forward_dim'] // 4,
        'Total Attention': arch_analysis['attention']['total_attention_capacity']
    }
    
    x = range(len(dims))
    ax2.bar(x, list(dims.values()))
    ax2.set_xticks(x)
    ax2.set_xticklabels(dims.keys(), rotation=45)
    ax2.set_ylabel('Dimension Size')
    ax2.set_title('Model Dimensions')
    ax2.grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    return dims

# Run architecture analysis
print("\n=== Architecture Analysis ===")
arch_analysis = analyze_architecture(configs['base'])

print("\nKey Architecture Metrics:")
print(f"Embedding dimension: {arch_analysis['architecture']['embedding_dim']}")
print(f"Number of layers: {arch_analysis['architecture']['num_layers']}")
print(f"Attention heads per layer: {arch_analysis['attention']['heads_per_layer']}")
print(f"Head dimension: {arch_analysis['attention']['head_capacity']}")
print(f"Feed-forward dimension: {arch_analysis['architecture']['feed_forward_dim']}")

print("\nAttention Analysis:")
print(f"Total attention capacity: {arch_analysis['attention']['total_attention_capacity']}")
print(f"Attention bottleneck ratio: {arch_analysis['attention']['attention_bottleneck_ratio']:.2f}")

print("\nDepth Analysis:")
print(f"Depth to width ratio: {arch_analysis['depth']['depth_to_width_ratio']:.2f}")
print(f"Parameter efficiency: {arch_analysis['depth']['parameter_efficiency']:.2f}")

# Plot attention patterns
dims = plot_attention_patterns(arch_analysis)

# Add scaling analysis
def analyze_scaling_characteristics(arch_analysis):
    """Analyze model scaling characteristics"""
    
    scaling_metrics = {
        'compute_to_params_ratio': arch_analysis['architecture']['num_layers'] * \
                                 (arch_analysis['architecture']['embedding_dim'] ** 2) / \
                                 (arch_analysis['architecture']['vocab_size'] * arch_analysis['architecture']['embedding_dim']),
        'attention_to_ffn_ratio': arch_analysis['attention']['total_attention_capacity'] / \
                                arch_analysis['architecture']['feed_forward_dim'],
        'depth_to_width_ratio': arch_analysis['depth']['depth_to_width_ratio']
    }
    
    scaling_recommendations = {
        'next_scale_factor': min(2.0, scaling_metrics['compute_to_params_ratio'] / 6.0),
        'suggested_width_scaling': scaling_metrics['attention_to_ffn_ratio'] > 0.25,
        'suggested_depth_scaling': scaling_metrics['depth_to_width_ratio'] < 1.0
    }
    
    return {
        'metrics': scaling_metrics,
        'recommendations': scaling_recommendations
    }

# Run scaling analysis
scaling_analysis = analyze_scaling_characteristics(arch_analysis)

print("\n=== Scaling Analysis ===")
print("\nScaling Metrics:")
for metric, value in scaling_analysis['metrics'].items():
    print(f"{metric}: {value:.2f}")

print("\nScaling Recommendations:")
for rec, value in scaling_analysis['recommendations'].items():
    print(f"{rec}: {value}")

In [ ]:
# Memory Efficiency Analysis
def analyze_memory_efficiency():
    """Analyze model memory usage and efficiency"""
    import torch
    
    def calculate_param_size(hidden_size, num_layers, vocab_size):
        # Embedding parameters
        embedding_params = vocab_size * hidden_size
        
        # Transformer layer parameters
        attention_params = 4 * hidden_size * hidden_size  # Q,K,V + output projection
        ffn_params = 4 * hidden_size * hidden_size  # Two linear layers
        layer_norm_params = 4 * hidden_size  # Two layer norms
        
        params_per_layer = attention_params + ffn_params + layer_norm_params
        total_layer_params = params_per_layer * num_layers
        
        # Total parameters
        total_params = embedding_params + total_layer_params
        return {
            'embedding': embedding_params,
            'attention': attention_params * num_layers,
            'ffn': ffn_params * num_layers,
            'layer_norm': layer_norm_params * num_layers,
            'total': total_params
        }
    
    # Calculate sizes for different precisions
    def get_size_in_mb(num_params, bytes_per_param):
        return (num_params * bytes_per_param) / (1024 * 1024)
    
    base_params = calculate_param_size(768, 12, 50000)
    
    memory_analysis = {
        'fp32_size': {k: get_size_in_mb(v, 4) for k,v in base_params.items()},
        'fp16_size': {k: get_size_in_mb(v, 2) for k,v in base_params.items()},
        'int8_size': {k: get_size_in_mb(v, 1) for k,v in base_params.items()}
    }
    
    # Activation memory estimation (batch_size=32, seq_len=512)
    batch_size = 32
    seq_len = 512
    hidden_size = 768
    
    def estimate_activation_memory():
        # Key activation sizes per layer
        attn_activations = batch_size * seq_len * hidden_size * 4  # Q,K,V + output
        ffn_activations = batch_size * seq_len * hidden_size * 4   # Two linear layers
        residual_activations = batch_size * seq_len * hidden_size
        
        total_per_layer = attn_activations + ffn_activations + residual_activations
        return get_size_in_mb(total_per_layer * 12, 2)  # Assuming FP16 training
    
    activation_mb = estimate_activation_memory()
    
    return memory_analysis, activation_mb

# Run memory analysis
memory_analysis, activation_mb = analyze_memory_efficiency()

print("\n=== Memory Efficiency Analysis ===")
print("\nModel Size by Precision:")
for precision, sizes in memory_analysis.items():
    print(f"\n{precision}:")
    for component, size in sizes.items():
        print(f"  {component}: {size:.2f} MB")

print(f"\nEstimated Activation Memory (batch=32, seq=512): {activation_mb:.2f} MB")

# Visualize memory distribution
def plot_memory_distribution(memory_analysis):
    """Plot memory usage distribution across components and precisions"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Component distribution for FP16
    sizes = memory_analysis['fp16_size']
    components = ['embedding', 'attention', 'ffn', 'layer_norm']
    sizes_list = [sizes[c] for c in components]
    
    ax1.pie(sizes_list, labels=components, autopct='%1.1f%%')
    ax1.set_title('Memory Distribution by Component (FP16)')
    
    # Precision comparison
    precisions = ['fp32_size', 'fp16_size', 'int8_size']
    total_sizes = [memory_analysis[p]['total'] for p in precisions]
    labels = ['FP32', 'FP16', 'INT8']
    
    ax2.bar(labels, total_sizes)
    ax2.set_ylabel('Size (MB)')
    ax2.set_title('Model Size by Precision')
    ax2.grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()

# Plot memory distribution
plot_memory_distribution(memory_analysis)

In [ ]:
# Transformer Architecture Analysis
def analyze_transformer_architecture():
    """Analyze transformer architecture components and interactions"""
    
    class TransformerBlockAnalysis:
        def __init__(self, hidden_size=768, num_heads=12, ff_dim=3072):
            self.hidden_size = hidden_size
            self.num_heads = num_heads
            self.head_dim = hidden_size // num_heads
            self.ff_dim = ff_dim
            
        def attention_complexity(self, seq_len):
            # Complexity of attention computation
            qk_multiply = seq_len * seq_len * self.head_dim * self.num_heads
            attn_multiply = seq_len * seq_len * self.head_dim * self.num_heads
            v_multiply = seq_len * self.hidden_size * self.hidden_size
            return {
                'qk_multiply': qk_multiply,
                'attention_multiply': attn_multiply,
                'value_multiply': v_multiply,
                'total': qk_multiply + attn_multiply + v_multiply
            }
            
        def ffn_complexity(self, seq_len):
            # Complexity of feed-forward computation
            first_layer = seq_len * self.hidden_size * self.ff_dim
            second_layer = seq_len * self.ff_dim * self.hidden_size
            return {
                'first_layer': first_layer,
                'second_layer': second_layer,
                'total': first_layer + second_layer
            }
            
        def receptive_field_analysis(self):
            return {
                'self_attention': 'Global (all positions)',
                'ffn': 'Local (position-wise)',
                'layer_norm': 'Local (position-wise)',
                'residual': 'Identity mapping'
            }
            
        def information_flow(self):
            return {
                'attention_bottleneck': self.head_dim,
                'ffn_bottleneck': self.ff_dim,
                'attention_paths': self.num_heads,
                'max_path_length': 'Linear in number of layers'
            }
    
    # Analyze model architecture
    model = TransformerBlockAnalysis()
    
    print("\n=== Transformer Architecture Analysis ===")
    
    # Analyze computational complexity
    seq_lengths = [128, 512, 1024]
    print("\nComputational Complexity Analysis:")
    for seq_len in seq_lengths:
        attn_complex = model.attention_complexity(seq_len)
        ffn_complex = model.ffn_complexity(seq_len)
        
        print(f"\nSequence Length: {seq_len}")
        print(f"Self-Attention Operations: {attn_complex['total']:,}")
        print(f"FFN Operations: {ffn_complex['total']:,}")
    
    # Analyze receptive field
    print("\nReceptive Field Analysis:")
    for component, field in model.receptive_field_analysis().items():
        print(f"  {component}: {field}")
    
    # Analyze information flow
    print("\nInformation Flow Analysis:")
    for metric, value in model.information_flow().items():
        print(f"  {metric}: {value}")
    
    return model

# Run architecture analysis
transformer_analysis = analyze_transformer_architecture()

# Visualize attention patterns
def plot_attention_patterns():
    """Visualize different attention pattern types"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    # Local attention pattern
    local = np.zeros((16, 16))
    for i in range(16):
        window = 3
        start = max(0, i-window)
        end = min(16, i+window+1)
        local[i, start:end] = 1
    axes[0,0].imshow(local)
    axes[0,0].set_title('Local Attention Pattern')
    
    # Global attention pattern
    global_attn = np.ones((16, 16))
    axes[0,1].imshow(global_attn)
    axes[0,1].set_title('Global Attention Pattern')
    
    # Causal attention pattern
    causal = np.tril(np.ones((16, 16)))
    axes[1,0].imshow(causal)
    axes[1,0].set_title('Causal Attention Pattern')
    
    # Sparse attention pattern
    sparse = np.zeros((16, 16))
    sparse[::2, ::2] = 1
    sparse[1::4, :] = 1
    axes[1,1].imshow(sparse)
    axes[1,1].set_title('Sparse Attention Pattern')
    
    for ax in axes.flat:
        ax.set_xlabel('Key Position')
        ax.set_ylabel('Query Position')
    
    plt.tight_layout()
    plt.show()

# Plot attention patterns
plot_attention_patterns()